<a href="https://colab.research.google.com/github/ddesai-sjsu/Data-Mining/blob/main/ANN_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Approximate Nerarest Search**

a) LSH

b) exhaustive search

c) product quantization

d) trees and graphs

e) hnsw

In [1]:
!pip install lightfm

     |████████████████████████████████| 310 kB 5.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705385 sha256=bfbaa614ac9c7236d842a43055cac7cd92b004cbf649f421e3943ad7af5dad5b
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


### **Creating Stack Exchange Dataset:**

Dataset used: Stack Exchange Dataset from LightFM

Finding questions similar to a given question in Stack Exchange.

In [2]:
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import pickle

In [3]:
stackExchange = fetch_stackexchange(dataset='crossvalidated',test_set_fraction=0.3)
train = stackExchange['train']
test = stackExchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackExchange['item_features'], epochs=20 )

item_vectors = stackExchange['item_features'] * model.item_embeddings

In [4]:
stackExchange['item_labels']=stackExchange['item_feature_labels']

In [5]:
with open('stackExchange.pickle', 'wb') as f:
    pickle.dump({"name": stackExchange['item_labels'], "vector": item_vectors}, f)

### **Locality Sensitive Hashing**

In [6]:
def load_data():
    with open('stackExchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[ 1.91029292e-02,  3.06971278e-02, -1.41312271e-01, ...,
          7.85037652e-02,  4.16649915e-02, -5.09055629e-02],
        [ 5.29068224e-02,  4.81603444e-02, -1.45479530e-01, ...,
          4.37527411e-02, -2.18923613e-02, -2.26521603e-04],
        [-7.72138461e-02, -1.34100250e-04, -1.88081101e-01, ...,
          5.02825417e-02,  1.04948692e-02, -1.24074958e-01],
        ...,
        [-9.68753546e-03,  7.53839910e-02,  4.37215194e-02, ...,
         -2.98996530e-02,  2.45406162e-02,  4.21907045e-02],
        [ 6.52044639e-02,  1.43514546e-02, -2.85745854e-03, ...,
          2.25938503e-02, -2.51188166e-02,  7.80131249e-03],
        [ 9.10495296e-02,  6.26801979e-03,  1.14080265e-01, ...,
         -8.21157265e-03, -5.29731959e-02,  5.87592367e-03]], dtype=float32)}

In [7]:
class FalconIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, self.dimention, number_of_partition, search_in_x_partitions, subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]
# https://github.com/erikbern/ann-benchmarks/commit/ecc56def165234fbec830fd1eed44396a1a52c49
# https://github.com/nmslib/nmslib/tree/master/python_bindings

In [18]:
!pip install faiss
!sudo apt-get install libopenblas-dev
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [19]:
import faiss

In [20]:
index = FalconIndex(data["vector"], data["name"])
index.build()

In [21]:
stackExchange_vector, stackExchange_name = data['vector'][90:91], data['name'][90]
similar_questions = '\n* '.join(index.query(stackExchange_vector))
print(f"The most similar questions to {stackExchange_name} are:\n* {similar_questions}")

The most similar questions to question_id:90 are:
* question_id:261
* question_id:5
* question_id:294
* question_id:3
* question_id:30
* question_id:50
* question_id:357
* question_id:340
* question_id:90
* question_id:67


### **Exhaustive Search**

In [22]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(self.vectors)
      
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [23]:
index = BruteForceIndex(data["vector"], data["name"])

In [24]:
stackExchange_vector, stackExchange_name = data['vector'][90:91], data['name'][90]
similar_questions = '\n* '.join(index.query(stackExchange_vector))
print(f"The most similar questions to {stackExchange_name} are:\n* {similar_questions}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:253
* question_id:930
* question_id:210
* question_id:810
* question_id:443
* question_id:67
* question_id:616
* question_id:596
* question_id:726


### **Product Quantization IVPQ**

In [25]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [26]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

In [27]:
stackExchange_index = 90
stackExchange_vector = data['vector'][stackExchange_index:stackExchange_index+1]
print(f"The most simillar questions to {data['name'][stackExchange_index]} are:")
index.query(stackExchange_vector)

The most simillar questions to question_id:90 are:


['question_id:261',
 'question_id:5',
 'question_id:294',
 'question_id:3',
 'question_id:30',
 'question_id:50',
 'question_id:357',
 'question_id:340',
 'question_id:90',
 'question_id:67']

### **Trees & Graphs**

In [28]:
!pip install annoy
!pip install  nmslib

     |████████████████████████████████| 646 kB 5.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391655 sha256=71666bbaa25de8496e6e32a9cf8e9711b6996dbaa3c69c44af6ecccb85d7a2de
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy
     |████████████████████████████████| 13.5 MB 4.3 MB/s 
     |████████████████████████████████| 188 kB 48.7 MB/s 


In [29]:
import annoy

In [30]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [31]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [32]:
stackExchange_vector, stackExchange_name = data['vector'][90], data['name'][90]
simlar_questions = '\n* '.join(index.query(stackExchange_vector))
print(f"The most similar questions to {stackExchange_name} are:\n* {simlar_questions}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:253
* question_id:5
* question_id:930
* question_id:616
* question_id:36
* question_id:131
* question_id:3451
* question_id:3118
* question_id:439


### **Hierarchical Navigable Small World Graphs**

In [33]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [34]:
import nmslib
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [35]:
stackExchange_vector, stackExchange_name = data['vector'][90], data['name'][90]
simlar_questions = '\n* '.join(index.query(stackExchange_vector))
print(f"The most similar questions to {stackExchange_name} are:\n* {simlar_questions}")

The most similar questions to question_id:90 are:
* question_id:90
* question_id:253
* question_id:5
* question_id:930
* question_id:810
* question_id:378
* question_id:22
* question_id:67
* question_id:210
* question_id:20748
